# Predicting Procurement Compliance Using KPI-Driven Machine Learning Models

**Author:** Brittany Dowdle

**Date:** 7/13/2025

**Objective:** This notebook is part of the overall [capstone project](https://github.com/Bdowdle4/Dowdle_Analytics_Capstone). The ultimate goal is to identify patterns and predictors of non-compliant suppliers using machine learning techniques.

## Introduction
This project uses the [Procurement KPI Analysis Dataset](https://www.kaggle.com/datasets/shahriarkabir/procurement-kpi-analysis-dataset) to predict compliance of suppliers. The dataset includes purchase order records from 5 different suppliers from 2022-2023. This notebook will be used to discover patterns, spot anomalies, test a hypothesis, and check assumptions to ensure the model will produce valid results.  

****

### Imports
In the code cell below are the necessary Python libraries for this notebook. *All imports should be at the top of the notebook.*

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

****
### 1. Load the Cleaned Dataset

In [3]:
# Load the dataset
df = pd.read_csv("C:/Users/Britt/Documents/44688/Dowdle_Analytics_Capstone/Data/cleaned_procurement_data.csv")

# Show the first few rows
print(df.head())

      po_id         supplier  order_date delivery_date    item_category  \
0  PO-00001        Alpha_Inc  2023-10-17    2023-10-25  Office Supplies   
1  PO-00002  Delta_Logistics  2022-04-25    2022-05-05  Office Supplies   
2  PO-00003         Gamma_Co  2022-01-26    2022-02-15              MRO   
3  PO-00004    Beta_Supplies  2022-10-09    2022-10-28        Packaging   
4  PO-00005  Delta_Logistics  2022-09-08    2022-09-20    Raw Materials   

  order_status  quantity  unit_price  negotiated_price  defective_units  \
0    Cancelled      1176       20.13             17.81              0.0   
1    Delivered      1509       39.32             37.34            235.0   
2    Delivered       910       95.51             92.26             41.0   
3    Delivered      1344       99.85             95.52            112.0   
4    Delivered      1180       64.07             60.53            171.0   

  compliance  lead_time_days  delivery_date_missing_flag  price_diff  \
0        Yes             8

### 2. Trend Line Charts